# IMPORT LIBRARY

In [1]:
import onnxruntime as rt
import numpy as np
import joblib
import os
import onnxmltools

## SETUP & LOAD

In [2]:
base_dir = os.getcwd()
model_path = os.path.join(base_dir, 'model', 'loan_model.onnx')
le_path = os.path.join(base_dir, 'model', 'label_encoder_v2.pkl')

# Load encoder
le = joblib.load(le_path)


In [3]:
# Load model ONNX 
sess = rt.InferenceSession(model_path)

# Get input name and output from metadata model
input_name = sess.get_inputs()[0].name
output_name = sess.get_outputs()[0].name

In [4]:
print(f"Model siap! Input name: '{input_name}'")

Model siap! Input name: 'float_input'


In [5]:
print(f"Model siap! Input name: '{output_name}'")

Model siap! Input name: 'label'


In [6]:
# Set Grade Mapping, because ONNX only can read numeric data
grade_map = {
    'A': 1.0,
    'B': 2.0,
    'C': 3.0,
    'D': 4.0,
    'E': 5.0,
    'F': 6.0,
    'G': 7.0
}

In [7]:
# test input data for check result model
input_amount = 15000
input_term = 24
input_grade = 'B'
input_rate = 0.12
input_installment = 700


In [9]:
try:
    grade_rank = grade_map[input_grade.upper()] # .upper() biar aman kalau user ketik huruf kecil
except KeyError:
    print("Error: Grade tidak valid! Harap masukkan A-G.")
    grade_rank = 4.0 # Default fallback (misal ke Grade D) jika error

print(f"Input User: {input_grade} --> Dikonversi ke Grade Rank: {grade_rank}")

Input User: B --> Dikonversi ke Grade Rank: 2.0


In [13]:
# input value data to model onnx 

data_input = np.array([[input_amount, input_term, grade_rank, input_rate, input_installment]]).astype(np.float32)

In [15]:
# prediction onnx 
pred_onx = sess.run([output_name], {input_name : data_input})[0] 
result_text = le.inverse_transform(pred_onx.astype(int))
print(f"Predicted Risk: {result_text[0]}")

Predicted Risk: 2
